In [1]:
import numpy as np
from PIL import Image
import os
import glob

In [2]:
# --- Configuration ---
SOURCE_DIR = r"D:\TEMP_DOWNLOAD\31_10_25_precise\31_10_25_precise"
MAIN_OUTPUT_DIR = r"D:\TEMP_DOWNLOAD\31_10_25_precise"

In [3]:
# 1. ค้นหาไฟล์ .npy ทั้งหมดใน SOURCE_DIR
npy_files = glob.glob(os.path.join(SOURCE_DIR, "*.npy"))

if not npy_files:
    print(f"ไม่พบไฟล์ .npy ใน: {SOURCE_DIR}")
    exit()

print(f"พบไฟล์ .npy ทั้งหมด {len(npy_files)} ไฟล์. เริ่มดำเนินการ...")

พบไฟล์ .npy ทั้งหมด 50 ไฟล์. เริ่มดำเนินการ...


In [4]:
# 2. วนลูปประมวลผลทีละไฟล์
for npy_file_path in npy_files:
    # ดึงชื่อไฟล์และสร้างชื่อโฟลเดอร์ผลลัพธ์
    base_name = os.path.basename(npy_file_path)
    folder_name = os.path.splitext(base_name)[0]  # เช่น '1tick_3'
    current_output_dir = os.path.join(MAIN_OUTPUT_DIR, folder_name)

    print(f"\n--- กำลังประมวลผล: {base_name} ---")

    # 3. สร้างโฟลเดอร์ผลลัพธ์สำหรับไฟล์นี้
    os.makedirs(current_output_dir, exist_ok=True)
    print(f"จะบันทึกผลลัพธ์ไปที่: {current_output_dir}")

    # 4. โหลด numpy array
    try:
        image_stack = np.load(npy_file_path)
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการโหลด '{npy_file_path}': {e}. ข้ามไฟล์นี้...")
        continue

    print(f"โหลดไฟล์แล้ว. Shape: {image_stack.shape}, Dtype: {image_stack.dtype}")

    # 5. ตรวจสอบ shape ของ array
    if image_stack.ndim not in [3, 4]:
        print(f"Error: Array ต้องเป็น 3D (N, H, W) หรือ 4D (N, H, W, C). ข้ามไฟล์นี้.")
        continue

    num_images = image_stack.shape[0]
    print(f"พบ {num_images} รูปภาพในไฟล์นี้...")

    # 6. วนลูป, แปลง, และบันทึก
    saved_count = 0
    for i in range(num_images):
        single_image_array = image_stack[i]

        # --- จัดการ Data Type (สำคัญมาก) ---
        if single_image_array.dtype != np.uint8:
            # print(f"Warning: Image {i} เป็น {single_image_array.dtype}, ไม่ใช่ uint8.")
            
            # กรณีเป็น float (เช่น 0.0 - 1.0)
            if np.issubdtype(single_image_array.dtype, np.floating):
                # print("ข้อมูลเป็น float, ทำการ scale 0.0-1.0 -> 0-255")
                # ตรวจสอบเผื่อว่าค่า max มันเกิน 1 (อาจจะเป็น float 0-255 อยู่แล้ว)
                max_val = np.max(single_image_array)
                if max_val <= 1.0:
                    single_image_array = (single_image_array * 255)
            
            # พยายามแปลงเป็น uint8
            try:
                single_image_array = single_image_array.astype(np.uint8)
            except ValueError:
                print(f"ไม่สามารถแปลง image {i}. ข้าม.")
                continue

        # 7. สร้าง PIL Image และบันทึก
        try:
            img = Image.fromarray(single_image_array)

            # กำหนดชื่อไฟล์ผลลัพธ์ (เช่น image_0000.png, image_0001.png)
            output_filename = os.path.join(current_output_dir, f'image_{i:04d}.png')

            # บันทึกรูปภาพ
            img.save(output_filename)
            saved_count += 1

        except ValueError as e:
            print(f"Error saving image {i}: {e}")
            print("มักเกิดจาก array shape ไม่ถูกต้อง (เช่น ข้อมูล RGB ใน 2D array).")
        except Exception as e:
            print(f"เกิดข้อผิดพลาดไม่คาดคิดกับ image {i}: {e}")

    print(f"บันทึกสำเร็จ {saved_count} / {num_images} รูปภาพ ไปยัง {current_output_dir}")

print("\n--- การแปลงไฟล์ทั้งหมดเสร็จสมบูรณ์ ---")


--- กำลังประมวลผล: 10tick_1.npy ---
จะบันทึกผลลัพธ์ไปที่: D:\TEMP_DOWNLOAD\31_10_25_precise\10tick_1
โหลดไฟล์แล้ว. Shape: (1560, 200, 200, 3), Dtype: uint16
พบ 1560 รูปภาพในไฟล์นี้...
บันทึกสำเร็จ 1560 / 1560 รูปภาพ ไปยัง D:\TEMP_DOWNLOAD\31_10_25_precise\10tick_1

--- กำลังประมวลผล: 10tick_2.npy ---
จะบันทึกผลลัพธ์ไปที่: D:\TEMP_DOWNLOAD\31_10_25_precise\10tick_2
โหลดไฟล์แล้ว. Shape: (1560, 200, 200, 3), Dtype: uint16
พบ 1560 รูปภาพในไฟล์นี้...
บันทึกสำเร็จ 1560 / 1560 รูปภาพ ไปยัง D:\TEMP_DOWNLOAD\31_10_25_precise\10tick_2

--- กำลังประมวลผล: 10tick_3.npy ---
จะบันทึกผลลัพธ์ไปที่: D:\TEMP_DOWNLOAD\31_10_25_precise\10tick_3
โหลดไฟล์แล้ว. Shape: (1560, 200, 200, 3), Dtype: uint16
พบ 1560 รูปภาพในไฟล์นี้...
บันทึกสำเร็จ 1560 / 1560 รูปภาพ ไปยัง D:\TEMP_DOWNLOAD\31_10_25_precise\10tick_3

--- กำลังประมวลผล: 10tick_4.npy ---
จะบันทึกผลลัพธ์ไปที่: D:\TEMP_DOWNLOAD\31_10_25_precise\10tick_4
โหลดไฟล์แล้ว. Shape: (1560, 200, 200, 3), Dtype: uint16
พบ 1560 รูปภาพในไฟล์นี้...
บันทึกสำเร็จ 1560